# Programación declarativa @ URJC
# Programación funcional
## Examen Convocatoria Extraordinaria
## Curso 19-20

# Definiciones auxiliares

### Algunas funciones sobre listas

In [ ]:
object Signatures{
    abstract class List[A]{
        
        // Common HOFs
        def foldRight[B](directSol: B)(composeSol: (A, B) => B): B
        def foldLeft[B](initial: B)(update: (B, A) => B): B
        def map[B](f: A => B): List[B]
        def flatMap[B](f: A => List[B]): List[B]
        def filter(f: A => Boolean): List[A]
        def forall(pred: A => Boolean): Boolean
        def exists(pred: A => Boolean): Boolean
                
        // Sort a list
        // e.g. List(3,2,1).sortWith((i, j) => i < j) == List(1,2,3)
        def sortWith(lt: (A, A) => Boolean): List[A]
        
        // Take the first `n` elements of the list
        // e.g. List(1,2,3).take(2) == List(1,2)
        //      List(1,2,3).take(0) == List()
        //      List(1,2,3).take(5) == List(1,2,3)
        def take(n: Int): List[A]
        
        // Return the head, if not empty
        def headOption: Option[A]
    }
}

### Definiciones sobre árboles binarios

In [ ]:
sealed abstract class Tree[A]
case class Empty[A]() extends Tree[A]
case class Node[A](left: Tree[A], root: A, right: Tree[A]) extends Tree[A]

In [ ]:
object Tree{
    
    def void[A]: Tree[A] = 
        Empty()
    
    def leaf[A](a: A): Node[A] = 
        Node(Empty(), a, Empty())
    
    def right[A](a: A, tree: Tree[A]): Node[A] = 
        Node(Empty(), a, tree)
    
    def left[A](tree: Tree[A], a: A): Node[A] = 
        Node(tree, a, Empty())
    
    def node[A](left: Tree[A], a: A, right: Tree[A]): Node[A] = 
        Node(left, a, right)
    
    def foldTree[A, B](tree: Tree[A])(empty: B)(node: (B, A, B) => B): B = 
        tree match {
            case Empty() => 
                empty
            case Node(left, a, right) =>
                node(foldTree(left)(empty)(node),
                    a,
                    foldTree(right)(empty)(node))
        }
}

import Tree._

### Modelo de datos de películas

In [ ]:
case class MovieDatabase(
    films: Map[Film.Id, Film],
    users: Map[User.Id, User],
    ratings: Map[(Film.Id, User.Id), Rating])
    
case class Film(
    id: Film.Id, 
    title: String, 
    director: String,
    genre: String,
    year: Int,
    country: String)

object Film{
    type Id = Int
}

case class User(
    id: User.Id,
    name: String,
    age: Int)
        
object User{
    type Id = Int
}
        
case class Rating(
    film: Film.Id,
    user: User.Id,
    score: Int)

In [ ]:
object BasicQueries{
    
    // Entities
    
    def films(mdb: MovieDatabase): List[Film] =
        mdb.films.values.toList
    
    def filmIds(mdb: MovieDatabase): List[Film.Id] =
        mdb.films.keys.toList

    def getFilm(id: Film.Id)(mdb: MovieDatabase): List[Film] = 
        mdb.films.get(id).toList
    
    def userIds(mdb: MovieDatabase): List[User.Id] = 
        mdb.users.keys.toList
    
    def getUser(id: User.Id)(mdb: MovieDatabase): List[User] = 
        mdb.users.get(id).toList
    
    // 1-N relationships
    
    def films(dir: String)(mdb: MovieDatabase): List[Film.Id] = 
        mdb.films.filter(_._2.director == dir).map(_._1).toList
    
    // N-M relationships
    
    def ratings(mdb: MovieDatabase): List[Rating] = 
        mdb.ratings.values.toList
    
    def userRatings(user: User.Id)(mdb: MovieDatabase): List[Rating] = 
        mdb.ratings.filter(_._1._2 == user).values.toList
    
    def filmRatings(film: Film.Id)(mdb: MovieDatabase): List[Rating] = 
        mdb.ratings.filter(_._1._1 == film).values.toList
}

import BasicQueries._